#Computing the volume underneath a Pareto frontier
This program computes the volume under a frontier of Pareto-optimal solutions to a multi-criterion optimization model. This volume provides a measure of the conflict among the objectives.

In [60]:
import pandas as pd
import numpy as np

### Preparation for the volume computation algorithm

In [61]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("solutionSets/2d/portfolioOptimization.csv")

In [62]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex"],axis=1) #remove solution index column

In [63]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [64]:
# Hard-coded manipulations to properly set objSenses

objSenses[" Return"] = 1 # set the senses on the objectives
objSenses[" Risk"] = 0 # 1 for max, and 0 for min

In [65]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.iteritems():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [66]:
# Compute the volume of the objective space

objSpaceVolume = 1
for obj,bounds in objBounds.iteritems():
    objSpaceVolume *= bounds[1] - bounds[0]

In [67]:
# Get the corner of the objective space that defines the ideal solution

idealSolution = []
for obj,bounds in objBounds.iteritems():
    idealSolution.append(bounds[objSenses[obj]])

In [68]:
# Get the corner of the objective space that defines the worst solution

worstSolution = []
for obj,bounds in objBounds.iteritems():
    worstSolution.append(bounds[not(objSenses[obj])])

### Algorithm to compute volume of frontier

#### Algorithm 1: "Add-the-sides" method
(Sandor)

In [70]:
# V = sum_{i \in N} A_i z_i
# where
# A_0 = 0
# and
# A_i = x_i y_i
    # - A_{i-1}
    # + \sum_{j<i : x_j > x_i} y_j \left[ x_j - \max_{l \le i}\{x_l : x_l < x_j \} \right]
    # + \sum_{j<i : y_j > y_i} x_j \left[ y_j - \max_{k \le i}\{y_k : y_k < y_j \} \right]

#### Algorithm 2: "Subtract-the-middle" method
(Nick) 

In [ ]:
# V = sum_{i \in N} A_i z_i
# where
# A_0 = 0
# and
# A_i = x_i y_i
    # - \sum_{s \in S_*} \min(x_*,x_s) \Delta_{y_s,y_{s-1}}
    # - x_{N^*} \left( y_* - \max_s\{ y_s : s \in S_* \} \right)

#### Algorithm 3: "Sort-first" method
(no one yet)

In [ ]:
# Sort objectives in some descending order, should simplify how to add N-1 dimensioned component of the solution,
# then multiply by value in Nth dimension to get contribution to polytope volume